In [26]:
from google.colab import drive
import pandas as pd
from IPython.display import display

drive.mount('/content/drive')

old_churches_1 = '/content/drive/MyDrive/OregonChurches/Old_Churches_Oregon_1.csv'
old_churches_2 = '/content/drive/MyDrive/OregonChurches/Old_Churches_Oregon_2.csv'
old_churches_3 = '/content/drive/MyDrive/OregonChurches/Old_Churches_Oregon_3.csv'
latest_churches = '/content/drive/MyDrive/OregonChurches/Churches_Latest.csv'

df_old_1 = pd.read_csv(old_churches_1)
df_old_2 = pd.read_csv(old_churches_2)
df_old_3 = pd.read_csv(old_churches_3)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
df_old_1.columns = df_old_1.columns.str.strip()
df_old_2.columns = df_old_2.columns.str.strip()
df_old_3.columns = df_old_3.columns.str.strip()

common_columns = ['Business Name', 'Entity Type', 'Nonprofit Type', 'Registry Date',
                  'Associated Name Type', 'Address', 'City', 'State', 'Zip Code']

common_columns_1 = [col for col in common_columns if col in df_old_1.columns]
common_columns_2 = [col for col in common_columns if col in df_old_2.columns]
common_columns_3 = [col for col in common_columns if col in df_old_3.columns]

df_old_1_filtered = df_old_1[common_columns_1]
df_old_2_filtered = df_old_2[common_columns_2]
df_old_3_filtered = df_old_3[common_columns_3]

df_old_merged = pd.concat([df_old_1_filtered, df_old_2_filtered, df_old_3_filtered], ignore_index=True)

df_latest = pd.read_csv(latest_churches)

df_latest.columns = df_latest.columns.str.strip()

df_latest_renamed = df_latest.rename(columns={
    'NAME': 'Business Name',
    'STREET': 'Address',
    'CITY': 'City',
    'STATE': 'State',
    'ZIP': 'Zip Code',
})

df_latest_filtered = df_latest_renamed[[col for col in common_columns if col in df_latest_renamed.columns]]
display(df_old_merged.head())
display(df_latest_filtered.head())

Columns of Old Churches 1:
Index(['Business Name', 'Entity Type', 'Nonprofit Type', 'Registry Date',
       'Associated Name Type', 'Address', 'Address Continued', 'City', 'State',
       'Zip Code'],
      dtype='object')

Columns of Old Churches 2:
Index(['Business Name', 'Entity Type', 'Registry Date', 'Nonprofit Type',
       'Address', 'City', 'State', 'Zip Code'],
      dtype='object')

Columns of Old Churches 3:
Index(['Registry Number', 'Business Name', 'Entity Type', 'Registry Date',
       'Nonprofit Type', 'Associated Name Type', 'First Name', 'Middle Name',
       'Last Name', 'Suffix', 'Not of Record Entity',
       'Entity of Record Reg Number', 'Entity of Record Name', 'Address',
       'Address Continued', 'City', 'State', 'Zip Code'],
      dtype='object')
Merged Old Churches:


,Business Name,Entity Type,Nonprofit Type,Registry Date,Associated Name Type,Address,City,State,Zip Code
0,THURSTON CHRISTIAN CHURCH,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1892-12-01 00:00:00,PRINCIPAL PLACE OF BUSINESS,6680 THURSTON RD,SPRINGFIELD,OR,97478
1,CENTENNIAL CHRISTIAN CENTER,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1897-04-23 00:00:00,PRINCIPAL PLACE OF BUSINESS,1020 JANUS ST,SPRINGFIELD,OR,97477
2,"FIRST BAPTIST CHURCH OF SPRINGFIELD, OREGON",DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1914-11-07 00:00:00,PRINCIPAL PLACE OF BUSINESS,1175 G ST,SPRINGFIELD,OR,97477
3,NEW LIFE CHURCH FIRST ASSEMBLY OF GOD SPRINGFIELD,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1940-12-30 00:00:00,PRINCIPAL PLACE OF BUSINESS,2080 N 19TH ST,SPRINGFIELD,OR,97477
4,FIRST CHRISTIAN CHURCH OF SPRINGFIELD,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1942-09-24 00:00:00,PRINCIPAL PLACE OF BUSINESS,395 W CENTENNIAL BLVD,SPRINGFIELD,OR,97477



Aligned Latest Churches:


,Business Name,Address,City,State,Zip Code
0,ZONTA CLUB OF GRANTS PASS OREGON FOUNDATION,PO BOX 1615,GRANTS PASS,OR,97528-0137
1,CANINE ACTION PACK,7375 SW LAKE BLUFF CT,WILSONVILLE,OR,97070-8466
2,EITING FOUNDATION,707 SW WASHINGTON ST STE 1500,PORTLAND,OR,97205-3532
3,KLAMATH COUNTY POLICE SERVICE DOG COMMUNITY SU...,PO BOX 1908,KLAMATH FALLS,OR,97601-0109
4,LOWER COLUMBIA SECTION OF THE PNCWA,5001 N COLUMBIA BLVD,PORTLAND,OR,97203-2098


In [32]:
remaining_churches = df_old_merged[~df_old_merged['Business Name'].isin(df_latest_filtered['Business Name'])]
remaining_churches_file = '/content/drive/MyDrive/OregonChurches/Remaining_Churches.csv'
remaining_churches.to_csv(remaining_churches_file, index=False)
remaining_churches_cleaned = remaining_churches.drop_duplicates(subset=['Business Name'])
remaining_churches_cleaned_file = '/content/drive/MyDrive/OregonChurches/Remaining_Churches_Cleaned.csv'
remaining_churches_cleaned.to_csv(remaining_churches_cleaned_file, index=False)
display(remaining_churches_cleaned.head())

churches_no_longer_exist = df_old_merged[~df_old_merged['Business Name'].isin(df_latest_filtered['Business Name'])]
churches_no_longer_exist = churches_no_longer_exist.drop_duplicates(subset=['Business Name'])
no_longer_exist_file = '/content/drive/MyDrive/OregonChurches/Churches_No_Longer_Exist.csv'
churches_no_longer_exist.to_csv(no_longer_exist_file, index=False)
display(churches_no_longer_exist.head())

Remaining Churches (in old but not in new):


,Business Name,Entity Type,Nonprofit Type,Registry Date,Associated Name Type,Address,City,State,Zip Code
0,THURSTON CHRISTIAN CHURCH,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1892-12-01 00:00:00,PRINCIPAL PLACE OF BUSINESS,6680 THURSTON RD,SPRINGFIELD,OR,97478
1,CENTENNIAL CHRISTIAN CENTER,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1897-04-23 00:00:00,PRINCIPAL PLACE OF BUSINESS,1020 JANUS ST,SPRINGFIELD,OR,97477
2,"FIRST BAPTIST CHURCH OF SPRINGFIELD, OREGON",DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1914-11-07 00:00:00,PRINCIPAL PLACE OF BUSINESS,1175 G ST,SPRINGFIELD,OR,97477
3,NEW LIFE CHURCH FIRST ASSEMBLY OF GOD SPRINGFIELD,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1940-12-30 00:00:00,PRINCIPAL PLACE OF BUSINESS,2080 N 19TH ST,SPRINGFIELD,OR,97477
5,"TRINITY BAPTIST CHURCH OF SPRINGFIELD, OREGON",DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1945-03-22 00:00:00,PRINCIPAL PLACE OF BUSINESS,1162 B ST,SPRINGFIELD,OR,97477



The remaining churches have been saved to: /content/drive/MyDrive/OregonChurches/Remaining_Churches.csv
Remaining Churches after removing duplicates based on Business Name:


,Business Name,Entity Type,Nonprofit Type,Registry Date,Associated Name Type,Address,City,State,Zip Code
0,THURSTON CHRISTIAN CHURCH,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1892-12-01 00:00:00,PRINCIPAL PLACE OF BUSINESS,6680 THURSTON RD,SPRINGFIELD,OR,97478
1,CENTENNIAL CHRISTIAN CENTER,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1897-04-23 00:00:00,PRINCIPAL PLACE OF BUSINESS,1020 JANUS ST,SPRINGFIELD,OR,97477
2,"FIRST BAPTIST CHURCH OF SPRINGFIELD, OREGON",DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1914-11-07 00:00:00,PRINCIPAL PLACE OF BUSINESS,1175 G ST,SPRINGFIELD,OR,97477
3,NEW LIFE CHURCH FIRST ASSEMBLY OF GOD SPRINGFIELD,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1940-12-30 00:00:00,PRINCIPAL PLACE OF BUSINESS,2080 N 19TH ST,SPRINGFIELD,OR,97477
5,"TRINITY BAPTIST CHURCH OF SPRINGFIELD, OREGON",DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1945-03-22 00:00:00,PRINCIPAL PLACE OF BUSINESS,1162 B ST,SPRINGFIELD,OR,97477



The cleaned remaining churches have been saved to: /content/drive/MyDrive/OregonChurches/Remaining_Churches_Cleaned.csv
Churches that no longer exist:


,Business Name,Entity Type,Nonprofit Type,Registry Date,Associated Name Type,Address,City,State,Zip Code
0,THURSTON CHRISTIAN CHURCH,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1892-12-01 00:00:00,PRINCIPAL PLACE OF BUSINESS,6680 THURSTON RD,SPRINGFIELD,OR,97478
1,CENTENNIAL CHRISTIAN CENTER,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1897-04-23 00:00:00,PRINCIPAL PLACE OF BUSINESS,1020 JANUS ST,SPRINGFIELD,OR,97477
2,"FIRST BAPTIST CHURCH OF SPRINGFIELD, OREGON",DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1914-11-07 00:00:00,PRINCIPAL PLACE OF BUSINESS,1175 G ST,SPRINGFIELD,OR,97477
3,NEW LIFE CHURCH FIRST ASSEMBLY OF GOD SPRINGFIELD,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1940-12-30 00:00:00,PRINCIPAL PLACE OF BUSINESS,2080 N 19TH ST,SPRINGFIELD,OR,97477
5,"TRINITY BAPTIST CHURCH OF SPRINGFIELD, OREGON",DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1945-03-22 00:00:00,PRINCIPAL PLACE OF BUSINESS,1162 B ST,SPRINGFIELD,OR,97477



The list of churches that no longer exist has been saved to: /content/drive/MyDrive/OregonChurches/Churches_No_Longer_Exist.csv
Churches that no longer exist (after removing duplicates):


,Business Name,Entity Type,Nonprofit Type,Registry Date,Associated Name Type,Address,City,State,Zip Code
0,THURSTON CHRISTIAN CHURCH,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1892-12-01 00:00:00,PRINCIPAL PLACE OF BUSINESS,6680 THURSTON RD,SPRINGFIELD,OR,97478
1,CENTENNIAL CHRISTIAN CENTER,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1897-04-23 00:00:00,PRINCIPAL PLACE OF BUSINESS,1020 JANUS ST,SPRINGFIELD,OR,97477
2,"FIRST BAPTIST CHURCH OF SPRINGFIELD, OREGON",DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1914-11-07 00:00:00,PRINCIPAL PLACE OF BUSINESS,1175 G ST,SPRINGFIELD,OR,97477
3,NEW LIFE CHURCH FIRST ASSEMBLY OF GOD SPRINGFIELD,DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1940-12-30 00:00:00,PRINCIPAL PLACE OF BUSINESS,2080 N 19TH ST,SPRINGFIELD,OR,97477
5,"TRINITY BAPTIST CHURCH OF SPRINGFIELD, OREGON",DOMESTIC NONPROFIT CORPORATION,RELIGIOUS WITH MEMBERS,1945-03-22 00:00:00,PRINCIPAL PLACE OF BUSINESS,1162 B ST,SPRINGFIELD,OR,97477



The list of churches that no longer exist has been saved to: /content/drive/MyDrive/OregonChurches/Churches_No_Longer_Exist.csv
